<a href="https://colab.research.google.com/github/Mayakshanesht/Bicycle_Tracking_project/blob/main/Yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
os.chdir("/content/drive/My Drive/YOLO")
!ls

coco.names			 mayur1.jpg	     Tutorial4
Data				 mayur_profile.jpg   yolov3.cfg
hungarian_tracking_algorithm.py  modified_yolov3.py  yolov3.py
Kalman_filter			 rohit.png	     yolov3.weights


In [115]:
!pip install opencv-python==4.5.2.52

In [116]:
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [117]:

import copy
import pickle
import time


In [118]:
#4lets write a class for object detection using YoloV3 with cv.dnn library
class YOLO():
  def __init__(self):
    self.nmsThreshold=0.7
    self.confThreshold=0.75
    self.inpHeight=416
    self.inpWidth=416
    self.classes=[]
    classes_file="/content/drive/My Drive/YOLO/Data/coco.names"
    with open(classes_file,'rt') as f:
      self.classes=f.read().rstrip('\n').split('\n')
    modelConfiguration="/content/drive/My Drive/YOLO/Data/yolov3.cfg"
    modelWeights="/content/drive/My Drive/YOLO/Data/yolov3.weights"
    self.i=0
    self.net=cv.dnn.readNetFromDarknet(modelConfiguration,modelWeights)
    self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
    self.net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

  def getOutputsNames(self):
    layerNames=self.net.getLayerNames()
    return [layerNames[i[0]-1] for i in self.net.getUnconnectedOutLayers()]
  
  def drawPred(self, frame, classId, conf, left, top, right, bottom):
    # Draw a bounding box.
    cv.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), thickness=5)
    label = '%.2f' % conf
    # Get the label for the class name and its confidence
    if self.classes:
        assert(classId < len(self.classes))
        label = '%s:%s' % (self.classes[classId], label)
    labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), thickness=3)
    return frame

  def postprocess(self,Frame,outs):
    FrameHeight=Frame.shape[0]
    FrameWidth=Frame.shape[1]
    classIds = []
    confidences = []
    boxes = []
    boxes1=[]
    global output_image
    for out in outs:
      for detections in out:
        scores=detections[5:]
        classId=np.argmax(scores)
        confidence=scores[classId]
        if ((confidence>self.confThreshold) and (classId==1)):
          center_x=int(detections[0]*FrameWidth)
          center_y=int(detections[1]*FrameHeight)
          width=int(detections[2]*FrameWidth)
          height=int(detections[3]*FrameHeight)
          left = int(center_x - width / 2)
          top = int(center_y - height / 2)
          classIds.append(classId)
          confidences.append(float(confidence))
          boxes.append([left, top, width, height])
     
     
     # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv.dnn.NMSBoxes(boxes, confidences, self.confThreshold, self.nmsThreshold)
    #print(indices)
    for i in indices:
        i = i[0]
        #print(i)
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        output_image = self.drawPred(Frame,classIds[i], confidences[i], left, top, left + width, top + height)
        box2=[left, top, left + width, top + height]
        boxes1.append(box2)
    return output_image,boxes1
  
  def inference(self,image):
      """
      Main loop.
      Input: Image
      Output: Frame with the drawn bounding boxes
      """
      # Create a 4D blob from a frame.
      blob = cv.dnn.blobFromImage(image, 1/255, (self.inpWidth, self.inpHeight), [0,0,0], 1, crop=False)
      # Sets the input to the network
      self.net.setInput(blob)
      # Runs the forward pass to get output of the output layers
      outs = self.net.forward(self.getOutputsNames())
      # Remove the bounding boxes with low confidence
      final_frame,BB = self.postprocess(image, outs)
      self.i +=1
      return final_frame,BB
    

In [119]:
detector = YOLO()
BICYCLE_MEASUREMENTS = []
TIMESTAMPS = []

def detection(image):
    # Run an inference using YOLOv3
    result_img, result_boxes = detector.inference(image)
    # Store the timestamp
    TIMESTAMPS.append(time.time())
    # If there is a bounding box, add the center
    if len(result_boxes) >0:
        BICYCLE_MEASUREMENTS.append([(result_boxes[0][0]+ result_boxes[0][2])/2, (result_boxes[0][1]+ result_boxes[0][3])/2])
    # Else, add "Not detected"
    else:
        BICYCLE_MEASUREMENTS.append("Not detected")
    return result_img

In [120]:
from moviepy.editor import VideoFileClip
video_file = "/content/drive/My Drive/YOLO/Kalman_filter/KF/images/VIRAT_S_010204_09_001285_001336.mp4"
clip = VideoFileClip(video_file).subclip(15,25)
white_clip = clip.fl_image(detection)
#cv.imshow(white_clip)
%time white_clip.write_videofile("/content/drive/My Drive/YOLO/Kalman_filter/KF/images/detect2.mp4",audio=False)

[MoviePy] >>>> Building video /content/drive/My Drive/YOLO/Kalman_filter/KF/images/detect2.mp4
[MoviePy] Writing video /content/drive/My Drive/YOLO/Kalman_filter/KF/images/detect2.mp4



100%|██████████| 240/240 [05:30<00:00,  1.38s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/YOLO/Kalman_filter/KF/images/detect2.mp4 

CPU times: user 10min 3s, sys: 6.46 s, total: 10min 10s
Wall time: 5min 31s


In [121]:
A=BICYCLE_MEASUREMENTS
print(BICYCLE_MEASUREMENTS)
T=TIMESTAMPS
print(TIMESTAMPS)

['Not detected', 'Not detected', 'Not detected', 'Not detected', [281.0, 322.5], [284.5, 323.5], [285.0, 324.0], [287.5, 324.0], [299.5, 326.5], [303.0, 327.0], [305.0, 326.5], [307.0, 327.0], [310.5, 326.5], [313.5, 323.5], [325.0, 323.5], [324.0, 326.5], [327.0, 327.0], [332.5, 327.0], [337.5, 323.5], [341.0, 324.0], [349.5, 325.5], [351.5, 325.5], [353.5, 325.0], [356.0, 325.0], [359.0, 325.0], [363.0, 325.0], [365.0, 322.5], [366.0, 322.5], [365.5, 322.0], [373.0, 325.5], [374.0, 325.0], [378.0, 324.5], [380.5, 323.5], [382.0, 323.5], [383.0, 324.0], [384.0, 323.0], [387.0, 322.0], [389.5, 322.0], 'Not detected', 'Not detected', [396.0, 322.5], [397.5, 323.5], [398.5, 323.0], [402.0, 322.5], [407.0, 336.0], [407.5, 335.5], [412.0, 336.0], [413.0, 336.5], [412.0, 339.5], 'Not detected', [421.5, 336.0], [422.0, 336.0], [424.0, 336.0], [428.5, 336.5], [430.0, 337.0], [434.0, 337.5], 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not de

In [122]:
BICYCLE_MEASUREMENTS = [[264.0, 320.5], 'Not detected', [276.5, 324.0], [278.0, 323.5], [281.0, 322.5], [284.5, 323.5], [285.0, 324.0], [287.5, 324.0], [299.5, 326.5], [303.0, 327.0], [305.0, 326.5], [307.0, 327.0], [310.5, 326.5], [313.5, 323.5], [325.0, 323.5], [324.0, 326.5], [327.0, 327.0], [332.5, 327.0], [337.5, 323.5], [341.0, 324.0], [349.5, 325.5], [351.5, 325.5], [353.5, 325.0], [356.0, 325.0], [359.0, 325.0], [363.0, 325.0], [365.0, 322.5], [366.0, 322.5], [365.5, 322.0], [373.0, 325.5], [374.0, 325.0], [378.0, 324.5], [380.5, 323.5], [382.0, 323.5], [383.0, 324.0], [384.0, 323.0], [387.0, 322.0], [389.5, 322.0], [390.0, 321.5], [395.0, 324.0], [396.0, 322.5], [397.5, 323.5], [398.5, 323.0], [402.0, 322.5], [407.0, 336.0], [407.5, 335.5], [412.0, 336.0], [413.0, 336.5], [412.0, 339.5], [420.5, 337.0], [421.5, 336.0], [422.0, 336.0], [424.0, 336.0], [428.5, 336.5], [430.0, 337.0], [434.0, 337.5], [434.5, 337.0], [437.5, 336.5], [438.5, 336.0], 'Not detected', 'Not detected', [451.5, 336.5], [454.0, 336.5], 'Not detected', 'Not detected', [460.0, 336.5], 'Not detected', [462.5, 336.0], 'Not detected', 'Not detected', [473.0, 336.0], 'Not detected', 'Not detected', 'Not detected', [483.0, 335.5], 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', [527.5, 351.0], 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', [596.0, 381.0], [597.0, 365.0], [600.0, 380.0], [605.0, 380.0], [607.0, 381.0], [608.0, 381.0], [612.0, 380.0], [612.0, 380.0], [619.0, 383.0], [620.0, 382.5], [624.0, 392.0], [625.5, 392.0], [628.0, 380.5], [630.0, 393.5], [633.0, 393.0], [635.0, 394.0], [637.0, 394.0], [637.0, 395.0], [644.0, 397.0], [644.0, 397.0], [646.0, 395.5], [647.5, 394.5], [651.5, 393.5], 'Not detected', [655.0, 407.0], [656.0, 391.0], 'Not detected', 'Not detected', 'Not detected', 'Not detected', [669.0, 410.0], [669.0, 410.0], [673.0, 419.5], [675.0, 422.5], [679.5, 422.5], [680.5, 424.5], [683.0, 424.5], [685.0, 426.0], [686.0, 432.0], [695.0, 433.5], [694.5, 434.5], [696.5, 437.0], [700.0, 421.5], [700.0, 421.5], [705.0, 421.5], [708.0, 423.0], [709.0, 424.5], [712.5, 427.0], [715.0, 426.5], [716.0, 429.5], [718.5, 431.5], [722.0, 434.0], [724.5, 433.5], [725.0, 434.5], 'Not detected', 'Not detected', 'Not detected', [744.0, 466.0], [745.5, 453.0], [747.5, 455.0], [749.5, 453.5], [752.0, 455.0], [750.5, 457.0], 'Not detected', 'Not detected', 'Not detected', [761.0, 475.5], [761.0, 475.5], [762.5, 475.5], [764.0, 478.0], 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', [805.5, 537.0], [809.5, 536.5], [811.0, 536.5], [815.0, 538.0], 'Not detected', 'Not detected', [825.5, 563.0], 'Not detected', [842.5, 567.0], [844.5, 571.0], [845.5, 588.5], [852.0, 589.5], [857.0, 591.5], [864.0, 621.0], [868.0, 622.0], [874.0, 624.0], [891.0, 620.5], [891.0, 620.5], [897.5, 622.5], [909.0, 651.0], [919.5, 651.0], [928.0, 626.5], 'Not detected', [948.0, 648.5], [955.5, 647.0], [971.5, 641.0], [976.5, 678.0], [993.0, 682.0], [1008.5, 685.0], [1008.5, 685.0], [1018.5, 688.0], [1048.5, 687.5], 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected', 'Not detected']
TIMESTAMPS = [1595343383.4528093, 1595343385.1370091, 1595343386.7868967, 1595343388.4632287, 1595343390.1011188, 1595343391.7460542, 1595343393.39343, 1595343395.028878, 1595343396.6552625, 1595343398.311383, 1595343399.9449534, 1595343401.5756452, 1595343403.253039, 1595343404.8913975, 1595343406.5174625, 1595343408.1842604, 1595343409.8111398, 1595343411.4432359, 1595343413.068713, 1595343414.6940196, 1595343416.3461204, 1595343417.9728732, 1595343419.6360095, 1595343421.2727242, 1595343422.934089, 1595343424.5932963, 1595343426.2158902, 1595343427.882905, 1595343429.5383394, 1595343431.1689167, 1595343432.7992053, 1595343434.4365535, 1595343436.093556, 1595343437.739813, 1595343439.3699863, 1595343441.026304, 1595343442.6605077, 1595343444.2896566, 1595343445.9262085, 1595343447.5629303, 1595343449.1896112, 1595343450.838177, 1595343452.4649684, 1595343454.2198577, 1595343455.8605254, 1595343457.4931364, 1595343459.137956, 1595343460.8218405, 1595343462.4845269, 1595343464.1445982, 1595343465.8397083, 1595343467.4892106, 1595343469.1747668, 1595343470.8333223, 1595343472.5270133, 1595343474.1908607, 1595343475.8525598, 1595343477.520469, 1595343479.1996396, 1595343480.9057856, 1595343482.604339, 1595343484.2830782, 1595343485.9620562, 1595343487.6216116, 1595343489.285687, 1595343490.9448197, 1595343492.599015, 1595343494.262499, 1595343495.9328613, 1595343497.604377, 1595343499.2598104, 1595343500.919045, 1595343502.5755124, 1595343504.2685933, 1595343505.9388247, 1595343507.6111808, 1595343509.727008, 1595343512.6301236, 1595343515.2081826, 1595343516.8960044, 1595343518.562933, 1595343520.223518, 1595343521.8825817, 1595343523.5388012, 1595343525.2359374, 1595343526.9006064, 1595343528.5632, 1595343530.250608, 1595343531.9128368, 1595343533.5599396, 1595343535.2325077, 1595343536.8938165, 1595343538.5590377, 1595343540.213533, 1595343541.8728576, 1595343543.534645, 1595343545.2209647, 1595343546.886572, 1595343548.5626056, 1595343550.2205744, 1595343551.8829527, 1595343553.5659046, 1595343555.2469, 1595343556.9008927, 1595343558.5652347, 1595343560.2281246, 1595343561.8948984, 1595343563.5591393, 1595343565.215573, 1595343566.8824282, 1595343568.5690107, 1595343570.2471654, 1595343571.9168723, 1595343573.5790687, 1595343575.2646444, 1595343576.9260058, 1595343578.6006463, 1595343580.262466, 1595343581.9326503, 1595343583.6091404, 1595343585.3043597, 1595343586.98645, 1595343588.677084, 1595343590.3740606, 1595343592.2494593, 1595343594.0968316, 1595343595.9687498, 1595343597.7663443, 1595343599.5230067, 1595343601.2006478, 1595343602.8708787, 1595343604.5619361, 1595343606.240997, 1595343607.9550593, 1595343609.640338, 1595343611.3281717, 1595343612.9923398, 1595343614.6791606, 1595343616.3586695, 1595343618.03751, 1595343619.7365143, 1595343621.4086063, 1595343623.0755863, 1595343624.7747118, 1595343626.4529173, 1595343628.1450953, 1595343629.8409429, 1595343631.5103605, 1595343633.2032287, 1595343634.8988585, 1595343636.5533113, 1595343638.2447634, 1595343639.9315574, 1595343641.6105409, 1595343643.3112786, 1595343644.9819617, 1595343646.6454144, 1595343648.3410013, 1595343650.0291924, 1595343651.7229924, 1595343653.4247396, 1595343655.0940385, 1595343656.766598, 1595343658.481635, 1595343660.1688697, 1595343661.851529, 1595343663.533179, 1595343665.2161155, 1595343666.9030716, 1595343668.6122663, 1595343670.3100102, 1595343671.9878461, 1595343673.6751158, 1595343675.361766, 1595343677.0522666, 1595343678.7405472, 1595343680.4148567, 1595343682.0903854, 1595343683.766348, 1595343685.4356513, 1595343687.1099372, 1595343688.82005, 1595343690.4986932, 1595343692.191855, 1595343693.8794398, 1595343695.5785794, 1595343697.2438045, 1595343698.998921, 1595343700.696096, 1595343702.375422, 1595343704.0798686, 1595343705.765661, 1595343707.4556317, 1595343709.2128184, 1595343710.8825693, 1595343712.5990374, 1595343714.2921202, 1595343715.9963896, 1595343717.6725974, 1595343719.3480031, 1595343721.0423503, 1595343722.7256033, 1595343724.4065895, 1595343726.0711443, 1595343727.7579536, 1595343729.4652514, 1595343731.1564894, 1595343732.8382416, 1595343734.516435, 1595343736.182922, 1595343737.882777, 1595343739.5695834, 1595343741.241596, 1595343742.9077015, 1595343744.5780733, 1595343746.2444737, 1595343747.9014044, 1595343749.56856, 1595343751.2365348, 1595343752.9167588, 1595343754.582135, 1595343756.2607112, 1595343757.9484925, 1595343759.6065814, 1595343761.2696397, 1595343762.9366505, 1595343764.5937672, 1595343766.266156, 1595343767.931009, 1595343769.606072, 1595343771.286105, 1595343772.9624963, 1595343774.622179, 1595343776.2862227, 1595343777.9396627, 1595343779.597795, 1595343781.266818, 1595343782.9147005, 1595343784.5958788, 1595343786.2695682, 1595343787.937123]

In [123]:
!pip install filterpy

In [124]:
from filterpy.kalman import KalmanFilter
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt

In [125]:
class CustomKalmanFilter():
  def __init__(self,dim_x,dim_z,dim_u=0):
    #dim_x=4
    #dim_z =2
    #dim_u = 0
    self.x = np.zeros(dim_x)
    self.P = np.eye(dim_x)*500
    Q_std = 0.01
    R_std = 10
    dt=float(sum(np.diff(TIMESTAMPS))/(len(TIMESTAMPS)))
    self.R = np.eye(dim_z)*R_std**2
    q = Q_discrete_white_noise(dim=dim_x/2,dt=dt,var=Q_std**2)
    self.Q = block_diag(q, q)
    self.F = np.array([[1,dt,0,0],
                     [0,1,0,0],
                     [0,0,1,dt],
                     [0,0,0,1]])
    self.H = np.array([[1,0,0,0],
                    [0,0,1,0]])
    self.y=np.zeros(dim_z)
    self.S=np.array([[0.,0.],[0.,0.]])
    self.K=np.zeros((dim_x,dim_z))
  
  def predict(self):
    self.x=np.dot(self.F,self.x)
    self.P=np.dot(np.dot(self.F,self.P),self.F.T)+self.Q

  def update(self,z):
    self.y=z-np.dot(self.H,self.x)
    self.S=np.dot(np.dot(self.H,self.P),self.H.T)+self.R
    self.K=np.dot(np.dot(self.P,self.H.T),np.linalg.inv(self.S))
    self.x+=np.dot(self.K,self.y)
    self.P=np.dot((np.eye(4)-np.dot(self.K,self.H)),self.P)

In [126]:
kf = CustomKalmanFilter(4,2,0)
kf.x = np.array([BICYCLE_MEASUREMENTS[0][0], 0, BICYCLE_MEASUREMENTS[0][1], 0])

In [129]:
ctr = 0
def main1(image):
    global kf
    global ctr
    meas = BICYCLE_MEASUREMENTS[ctr] 
    if meas == "Not detected":
        kf.predict()
        cv.circle(image, (int(kf.x[0]), int(kf.x[2])), 5, (255,0,0), -1)
    else:
        z = np.array((meas[0], meas[1]))
        kf.update(z)
        cv.circle(image, (int(kf.x[0]), int(kf.x[2])), 5, (0,0,255), -1)
        kf.predict()
        cv.circle(image, (int(kf.x[0]), int(kf.x[2])), 5, (255,0,0), -1)
    ctr+=1
    return image

In [130]:
from moviepy.editor import VideoFileClip
video_file = "/content/drive/My Drive/YOLO/Kalman_filter/KF/images/detect2.mp4"
clip = VideoFileClip(video_file)
tracked_clip = clip.fl_image(main1)
%time tracked_clip.write_videofile("/content/drive/My Drive/YOLO/Kalman_filter/KF/images/track5.mp4",audio=False)

[MoviePy] >>>> Building video /content/drive/My Drive/YOLO/Kalman_filter/KF/images/track5.mp4
[MoviePy] Writing video /content/drive/My Drive/YOLO/Kalman_filter/KF/images/track5.mp4



100%|█████████▉| 240/241 [00:11<00:00, 20.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/YOLO/Kalman_filter/KF/images/track5.mp4 

CPU times: user 2.27 s, sys: 6.23 s, total: 8.5 s
Wall time: 12.8 s
